In [ ]:
import pandas as pd
from tabulate import tabulate
from scipy.stats import ttest_ind
import warnings
pd.set_option('display.max_columns', None)

In [ ]:
play2024_df = pd.read_csv("data/play_by_play_2024.csv")
injury2024_df = pd.read_csv("data/injuries_2024.csv")

In [ ]:
def first_last_a(name):
    parts = name.split(" ", 1)
    return f"{parts[0][0]}.{parts[1]}"

In [ ]:
def first_last_b(name):
    parts = name.split(" ", 1)
    return f"{parts[0][0:2]}.{parts[1]}"

In [ ]:
warnings.filterwarnings('ignore')

def merge_play_injury_dfs(play_df, injury_df):

    # preprocessing
    injury_df["date"] = pd.to_datetime(injury_df['date_modified'])
    play_df["date"] = pd.to_datetime(play_df['game_date'])
    injury_df['date'] = injury_df['date'].dt.tz_localize(None)
    play_df['date'] = play_df['date'].dt.tz_localize(None)

    # filtering
    plays_with_injuries = play_df[play_df['desc'].str.contains("was injured", na=False)]
    pattern = r'(\w+\.(?:\w|-|\.|\')+(?: \w+)*) was injured'
    # Extract the injured player's name from the desc column
    injured_players = plays_with_injuries.loc[:, "desc"].str.extract(pattern)

    # concatenation
    plays_with_injuries = pd.concat([plays_with_injuries, injured_players], axis=1)
    plays_with_injuries.rename(columns={0: "injured_player"}, inplace=True)
    plays_with_injuries = plays_with_injuries.reset_index(drop=True)
    
    # merging
    injuries = []
    for (week, team), group_injury_df in injury_df.groupby(['week', 'team']):
        group_play_df = plays_with_injuries[(plays_with_injuries['week'] == week) & ((plays_with_injuries['home_team'] == team) | (plays_with_injuries['away_team'] == team))]

        group_injury_df = group_injury_df[group_injury_df.date >= group_play_df.date.max()]

        group_injury_df["first_type"] = group_injury_df['full_name'].apply(first_last_a)
        group_injury_df["second_type"] = group_injury_df['full_name'].apply(first_last_b)

        x = pd.merge(group_play_df, group_injury_df, left_on="injured_player", right_on="first_type", how="inner")
        y = pd.merge(group_play_df, group_injury_df, left_on="injured_player", right_on="second_type", how="inner")

        injuries.append(pd.concat([x, y], axis = 0, ignore_index=True))

    plays_with_injuries_and_injury_record = (pd.concat(injuries, axis=0, ignore_index=True)).drop(columns=["first_type", "second_type"])
    plays_with_injuries_and_injury_record = plays_with_injuries_and_injury_record.sort_values('play_id', ascending=False).drop_duplicates(subset=['week_x', 'full_name', "team"], keep='first')

    return plays_with_injuries, plays_with_injuries_and_injury_record
#returns (plays where injuries occurred, plays were injuries occurred and missed time)



In [ ]:
def populate_cols_in_play_df(play_df, plays_with_injuries, plays_with_injuries_and_injury_record):
    columns_to_check = ['play_id', 'game_id']
    play_df["was_injured"] = 0
    play_df["missed_time"] = 0
    play_df.loc[play_df[columns_to_check].apply(tuple, 1).isin(plays_with_injuries[columns_to_check].apply(tuple, 1)), 'was_injured'] = 1
    play_df.loc[play_df[columns_to_check].apply(tuple, 1).isin(plays_with_injuries_and_injury_record[columns_to_check].apply(tuple, 1)), 'missed_time'] = 1

    return play_df

In [ ]:
plays_with_injuries_2024, plays_with_injuries_and_injury_record_2024 = merge_play_injury_dfs(play2024_df, injury2024_df)


In [ ]:
# Print number of rows for each returned table
print("Rows in plays_with_injuries_2024:", len(plays_with_injuries_2024))
print("Rows in plays_with_injuries_and_injury_record_2024:", len(plays_with_injuries_and_injury_record_2024))


In [ ]:
play2024_df = populate_cols_in_play_df(play2024_df, plays_with_injuries_2024, plays_with_injuries_and_injury_record_2024)
print(len(play2024_df))
print(len(play2024_df.columns.tolist()))
print(play2024_df.columns.tolist())


### MODELING 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

In [127]:
# Drop unnecessary columns from X
X = play2024_df.drop(columns=[
    'was_injured', 'missed_time', 'play_id', 'game_id', 'game_date', 'desc', 
    'td_player_name', 'passer_player_name', 'rusher_player_name', 'receiver_player_name', 
    'nfl_api_id', 'fantasy_player_name', 'fantasy_player_id', 'passer_jersey_number', 
    'rusher_jersey_number', 'receiver_jersey_number', 'jersey_number'
])

# Drop all columns with datetime64 data type from X
X = X.select_dtypes(exclude=['datetime64'])

# Define target variables
y_injury = play2024_df['was_injured']
y_severe_injury = play2024_df['missed_time']

# Separate numerical and categorical columns
numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Fill missing values for numerical columns with the mean
X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].mean())

# Fill missing values for categorical columns with 'Unknown'
X[categorical_cols] = X[categorical_cols].fillna('Unknown')

# Encode categorical variables
label_encoders = {}
for column in categorical_cols:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le  # Store encoders for future use

# Split the data into training and test sets
X_train, X_test, y_train_injury, y_test_injury = train_test_split(X, y_injury, test_size=0.3, random_state=42)
_, _, y_train_severe, y_test_severe = train_test_split(X, y_severe_injury, test_size=0.3, random_state=42)

# Re-check and fill any remaining NaN values in X_train
if X_train.isnull().any().any():
    print("NaN values found in X_train; filling missing values.")
    # Fill any remaining NaNs in numerical columns with the mean
    X_train[numerical_cols] = X_train[numerical_cols].fillna(X_train[numerical_cols].mean())
    # Fill any remaining NaNs in categorical columns with 'Unknown'
    X_train[categorical_cols] = X_train[categorical_cols].fillna('Unknown')

# Apply SMOTE to the training sets to balance the classes
smote = SMOTE(random_state=42)
X_train_injury_balanced, y_train_injury_balanced = smote.fit_resample(X_train, y_train_injury)
X_train_severe_balanced, y_train_severe_balanced = smote.fit_resample(X_train, y_train_severe)

# Compute class weights based on the training labels
class_weights_injury = class_weight.compute_sample_weight('balanced', y_train_injury_balanced)
class_weights_severe = class_weight.compute_sample_weight('balanced', y_train_severe_balanced)

# Train the Random Forest for predicting injury
rf_injury = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_injury.fit(X_train_injury_balanced, y_train_injury_balanced, sample_weight=class_weights_injury)

# Train the Random Forest for predicting severe injury
rf_severe = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_severe.fit(X_train_severe_balanced, y_train_severe_balanced, sample_weight=class_weights_severe)

# Predict on the test set
y_pred_injury = rf_injury.predict(X_test)
y_pred_severe = rf_severe.predict(X_test)

# Print classification report for both models
print("Injury Prediction Report")
print(classification_report(y_test_injury, y_pred_injury, target_names=['No Injury', 'Injury']))

print("\nSevere Injury Prediction Report")
print(classification_report(y_test_severe, y_pred_severe, target_names=['No Missed Time', 'Missed Time']))

# Feature importance
injury_importances = pd.Series(rf_injury.feature_importances_, index=X.columns).sort_values(ascending=False)
severe_importances = pd.Series(rf_severe.feature_importances_, index=X.columns).sort_values(ascending=False)

# Display the top important features for both models
print("\nTop Features for Injury Prediction:")
print(injury_importances.head(10))

print("\nTop Features for Severe Injury Prediction:")
print(severe_importances.head(10))


NaN values found in X_train; filling missing values.


ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X = play2024_df.drop(columns=[
    'was_injured', 'missed_time', 'play_id', 'game_id', 'game_date', 'desc', 
    'td_player_name', 'passer_player_name', 'rusher_player_name', 'receiver_player_name', 
    'nfl_api_id', 'fantasy_player_name', 'fantasy_player_id', 'passer_jersey_number', 'rusher_jersey_number',
    'receiver_jersey_number', 'jersey_number'
])

# Drop all columns with datetime64 data type from X
X = X.select_dtypes(exclude=['datetime64'])

y_injury = play2024_df['was_injured']
y_severe_injury = play2024_df['missed_time']

# Separate numerical and categorical columns
numerical_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Fill missing values for numerical columns with the mean
X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].mean())

# Fill missing values for categorical columns with 'Unknown'
X[categorical_cols] = X[categorical_cols].fillna('Unknown')

# Encode categorical variables
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le  # Store encoders for future use

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train_injury, y_test_injury = train_test_split(X, y_injury, test_size=0.3, random_state=42)
_, _, y_train_severe, y_test_severe = train_test_split(X, y_severe_injury, test_size=0.3, random_state=42)

# Compute class weights based on the training labels, not the entire dataset
class_weights_injury = class_weight.compute_sample_weight('balanced', y_train_injury)
class_weights_severe = class_weight.compute_sample_weight('balanced', y_train_severe)



In [ ]:
# Train the Random Forest for predicting injury
rf_injury = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_injury.fit(X_train, y_train_injury, sample_weight=class_weights_injury)


# Train the Random Forest for predicting severe injury
rf_severe = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_severe.fit(X_train, y_train_severe, sample_weight=class_weights_severe)


In [ ]:
# Predict on the test set
y_pred_injury = rf_injury.predict(X_test)
y_pred_severe = rf_severe.predict(X_test)

# Print classification report for both models
print("Injury Prediction Report")
print(classification_report(y_test_injury, y_pred_injury, target_names=['No Injury', 'Injury']))

print("\nSevere Injury Prediction Report")
print(classification_report(y_test_severe, y_pred_severe, target_names=['No Missed Time', 'Missed Time']))

# Feature importance
injury_importances = pd.Series(rf_injury.feature_importances_, index=X.columns).sort_values(ascending=False)
severe_importances = pd.Series(rf_severe.feature_importances_, index=X.columns).sort_values(ascending=False)

# Display the top important features for both models
print("\nTop Features for Injury Prediction:")
print(injury_importances.head(10))

print("\nTop Features for Severe Injury Prediction:")
print(severe_importances.head(10))
